# Convolutional Neural Network
In this tutorial, we will implement a convolutional neural network to classify mnist dataset.

## Import library and load MNIST dataset
MNIST dataset has 55,000 examples for training and 10,000 examples for testing. Each image has a size of 784 (28x28).

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
from ops import conv2d, linear


mnist = input_data.read_data_sets("mnist/", one_hot=True)
x_train = mnist.train.images.reshape(-1, 28, 28, 1)
y_train = mnist.train.labels
x_test = mnist.test.images.reshape(-1, 28, 28, 1)
y_test = mnist.test.labels

print "x_train: ", x_train.shape
print "y_train: ", y_train.shape
print "x_test: ", x_test.shape
print "y_test: ", y_test.shape

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz
x_train:  (55000, 28, 28, 1)
y_train:  (55000, 10)
x_test:  (10000, 28, 28, 1)
y_test:  (10000, 10)


## Parameter settings 

In [2]:
num_class = 10

params = {
    # weights and biases for convolution layer
    'w_conv1': tf.Variable(tf.random_normal([5, 5, 1, 8], stddev=0.1)),
    'b_conv1': tf.Variable(tf.zeros([8])),
    'w_conv2': tf.Variable(tf.random_normal([5, 5, 8, 16], stddev=0.1)),
    'b_conv2': tf.Variable(tf.zeros([16])),
    
    # weights and biases for fully-connected layer
    'w_h': tf.Variable(tf.random_normal([7*7*16, 64], stddev=0.1)),
    'b_h': tf.Variable(tf.zeros([64])),
    'w_out': tf.Variable(tf.random_normal([64, num_class], stddev=0.1)),
    'b_out': tf.Variable(tf.zeros([num_class]))
}

## Define and construct model

Now, we will develope a convolutional neural network with 2 convolution layers and 1 hidden fully connected layer.

In [3]:
def convolutional_network(x):
    # 1st convolution layer
    conv1 = tf.nn.conv2d(x, params['w_conv1'], strides=[1,1,1,1], padding='SAME') + params['b_conv1']
    conv1 = tf.nn.relu(conv1)
    conv1 = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    # 2nd convolution layer
    conv2 = tf.nn.conv2d(conv1, params['w_conv2'], strides=[1,1,1,1], padding='SAME') + params['b_conv2']
    conv2 = tf.nn.relu(conv2)
    conv2 = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    # fully connected layer
    conv2 = tf.reshape(conv2, [tf.shape(conv2)[0], -1])
    h = tf.matmul(conv2, params['w_h']) + params['b_h']
    h = tf.nn.relu(h)
    out = h = tf.matmul(h, params['w_out']) + params['b_out']
    
    return out

In [4]:
# construct model
x = tf.placeholder(dtype=tf.float32, shape=[None, 28, 28, 1])
y = tf.placeholder(dtype=tf.int64, shape=[None, 10])
out = convolutional_network(x)

# define loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(out, y))
train_op = tf.train.RMSPropOptimizer(learning_rate=0.01).minimize(loss)

# find correct prediction and accuracy
correct_pred = tf.equal(tf.argmax(out,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

## Train and test the model

From above, we build our neural network model to classify the MNIST dataset. To launch our model, we will implement a session where the model is actually trained and tested on the MNIST dataset.
#### Train
First, we initialize all variables we created above. This can be done by running `tf.initialize_all_variables()`. The most important part of code in the training phase is `sess.run([train_op, loss], feed_dict={x: x_batch, y:y_batch})`. This part of code feeds mini-batch data into placeholder and run train_op to update variables with `tf.train.RMSPropOptimizer` once. Also, loss is evaluated to print out the average loss for each epoch.
#### Test
Testing phase is quite simple. We use `sess.run(accuracy, feed_dict={x: x_batch, y: y_batch})` to print out the test accuracy.

In [6]:
batch_size = 100
num_epoch = 5
num_iter_per_epoch = int(x_train.shape[0] / batch_size)

# launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
    # initialize tensor variables
    tf.initialize_all_variables().run()
    
    # training cycle
    for epoch in range(num_epoch):
        avg_loss = 0.
        # loop over all batches
        for i in range(num_iter_per_epoch):
            x_batch = x_train[i*batch_size:(i+1)*batch_size]
            y_batch = y_train[i*batch_size:(i+1)*batch_size]
        
            # run optimization op (backprop) and loss op (to get loss value)
            _, c = sess.run([train_op, loss], feed_dict={x: x_batch, y: y_batch})
            
            # compute average loss
            avg_loss += c / num_iter_per_epoch
        print ("Epoch %d, Loss: %.3f"% (epoch+1, avg_loss))
    print ("Finished training!\n")
    
    
    num_iter_per_epoch = int(x_test.shape[0] / batch_size)
    test_accuracy = 0.0
    for i in range(num_iter_per_epoch):
        x_batch = x_test[i*batch_size:(i+1)*batch_size]
        y_batch = y_test[i*batch_size:(i+1)*batch_size]
       
        acc = sess.run(accuracy, feed_dict={x: x_batch, y: y_batch})
        test_accuracy += acc

    print ("Test accuracy: %.3f" %(test_accuracy/num_iter_per_epoch))

Epoch 1, Loss: 0.379
Epoch 2, Loss: 0.074
Epoch 3, Loss: 0.067
Epoch 4, Loss: 0.063
Epoch 5, Loss: 0.057
Finished training!

Test accuracy: 0.973
